##Wikipedia API

In [ ]:
from wikipedia import Wikipedia
from wiki2plain import Wiki2Plain

In [4]:
import numpy as np
import pandas as pd

In [5]:
import ast
import json
import requests

In [6]:
# print_function turns print into a python 3 function 
# this enables python to print funky text i.e. f\xn --> fun
from __future__ import (division,
                        absolute_import,
                        print_function,
                        unicode_literals
                        )

##Corpus Curation - Wiki

####Get Math Topics from Wiki

In [4]:
# THIS CODE NO LONGER NEEDS TO RUN

headers = {'user_agent': 'DataWrangling/1.1 (http://galvanize.it; class@galvanize.it)'}
page = 'Lists_of_mathematics_topics'
payload = 'action=parse&format=json&page=' + page
url = "https://en.wikipedia.org/w/api.php?"
r = requests.post(url, data=payload, headers=headers)
topic_pages = r.json()['parse']['links']

corpus_pages = []
for i,page_complex in enumerate(topic_pages):
    # This list will be what we use to scrape, so if it is just a "list of" page
    # we are going to skip it for now
    if not page_complex['*'].startswith(u'List'):
        payload = 'action=parse&format=json&page=' + page_complex['*']
        url = "https://en.wikipedia.org/w/api.php?"
        try:
            r = requests.post(url, data=payload, headers=headers)
            corpus_pages.extend(r.json()['parse']['links'])
        except UnicodeEncodeError:
            print('Error getting: ' + page_complex['*'])
            continue
        if i % 10 == 0 or i == len(topic_pages):
            print('Processing ' + str(i) + ' of ' + str(len(topic_pages)))
        
print('Total scrapped pages: ' + str(len(corpus_pages)))

In [5]:
# THIS CODE NO LONGER NEEDS TO RUN

# # import 2nd half of corpus starting from 33530 +1090 +39530
# # upload file to corpus
# lang = 'en' # standard english wiki pages
# wiki = Wikipedia(lang)
# for i, page in enumerate(corpus_pages[33530+1090:]):
#     try: 
#         subject = page["*"]
#         raw = wiki.article(subject)
#         wiki2plain = Wiki2Plain(raw)
#         content = wiki2plain.text
#         url = 'https://gateway.watsonplatform.net/concept-insights/api/v2/corpora/liet4blerinu2/math_corp/documents/' + subject

#     #     payload = {"label": subject,
#     #                "parts": {"data": content,
#     #                           "name": subject,
#     #                           "content-type": "text"},
#     #                "user_fields": {"myField1": "my meta-data"}}


#         payload = {"label": subject,
#                "parts": [{"data": content,
#                           "name": "article",
#                           "content-type": "text"},
#                          {"data": "Part of test document pure text."
#                           ,"name": "Text part",
#                           "content-type": "text/plain"}],
#                "user_fields": {"myField1": "my meta-data"}}

#         r2 = requests.post(url = url,
#                           data = json.dumps(payload),
#                           auth=auth)
#     except:
#         print ("some error", "file", i,  subject)
        
#     if r2.status_code != 201:
#         print (r2.status_code, subject)
#     if i % 10 == 0:
#         print (i)

In [14]:
# starter code from StackOverFlow
# import os.path

# def scrape_wiki(path,topic):
#     '''
#     Get all text from wiki page based on the topic entered.
#     '''
#     lang = 'en' # standard english wiki pages
#     wiki = Wikipedia(lang)
        
#     fname = path + topic + '.txt'
    
#     if not os.path.isfile(fname):    
#         try:
#             raw = wiki.article(topic)
#         except:
#             raw = None

#         if raw:
#             wiki2plain = Wiki2Plain(raw)
#             content = wiki2plain.text

#             with open(fname,'w') as handle:
#                 handle.write(content) 

##Concept Insights (Explore APIs)

###Get Account ID

In [7]:
from pprint import pprint

In [8]:
# get account_id
base_url = "https://watson-api-explorer.mybluemix.net/concept-insights/api"
account_url = 'https://watson-api-explorer.mybluemix.net/concept-insights/api/v2/accounts'
auth=('82a698f0-8d8c-42f9-9c7d-1e4b97eebd52','50KJG6jdgZ6h')

r = requests.get(account_url, auth=auth)

In [9]:
# display account id
print(r.content)

{"accounts":[{"account_id":"liet4blerinu2"}]}


###View Document Annotations

In [64]:
# GET /v2/corpora/{account_id}/{corpus}/documents/{document}/annotations
url = 'https://gateway.watsonplatform.net/concept-insights/api/v2/corpora/liet4blerinu2/math_corp/documents/test/annotations'
r = requests.get(url,auth=auth)

print (r.status_code)

ast.literal_eval(r.content)

200


{'annotations': [],
 'id': '/corpora/liet4blerinu2/math_corp/documents/test',
 'label': 'test'}

###Get available corpora

In [9]:
available_corpora = 'https://gateway.watsonplatform.net/concept-insights/api/v2/corpora/liet4blerinu2'

r = requests.get(available_corpora, auth=auth)
r.status_code

200

In [10]:
# display available corpora and it's parameters
ast.literal_eval(r.content)

{'corpora': [{'access': 'private',
   'id': '/corpora/liet4blerinu2/math_corp',
   'users': [{'account_id': '', 'permission': 'ReadWriteAdmin'},
    {'account_id': 'liet4blerinu2', 'permission': 'ReadWriteAdmin'}]}]}

###Delete Corpus by ID

In [18]:
# DELETE /v2/corpora/{account_id}/{corpus} Deletes a corpus by ID
url = 'https://gateway.watsonplatform.net/concept-insights/api/v2/corpora/liet4blerinu2/math_corp2'
requests.delete(url)

<Response [401]>

In [22]:
# DELETE /v2/corpora/{account_id}/{corpus} Deletes a corpus by ID
!curl -u "82a698f0-8d8c-42f9-9c7d-1e4b97eebd52":'50KJG6jdgZ6h' \
        -X DELETE 'https://gateway.watsonplatform.net/concept-insights/api/v2/corpora/liet4blerinu2/math_corp2'

### Retrive document id's

In [32]:
# GET /v2/corpora/{account_id}/{corpus}/documents
limit = '100' 
url = 'https://gateway.watsonplatform.net/concept-insights/api/v2/corpora/liet4blerinu2/math_corp/documents?limit=' + limit
r = requests.get(url,auth=auth)

print (r.status_code)

200


In [33]:
ast.literal_eval(r.content)

{'documents': ['/corpora/liet4blerinu2/math_corp/documents/Calculus',
  '/corpora/liet4blerinu2/math_corp/documents/Trigonometry',
  '/corpora/liet4blerinu2/math_corp/documents/Geometry',
  '/corpora/liet4blerinu2/math_corp/documents/Differential%20Equations',
  '/corpora/liet4blerinu2/math_corp/documents/Integrals',
  '/corpora/liet4blerinu2/math_corp/documents/Unit%20Circle',
  '/corpora/liet4blerinu2/math_corp/documents/Sine',
  '/corpora/liet4blerinu2/math_corp/documents/Cosine',
  '/corpora/liet4blerinu2/math_corp/documents/pythagorean%20theorem',
  '/corpora/liet4blerinu2/math_corp/documents/Circle',
  '/corpora/liet4blerinu2/math_corp/documents/Circumference',
  '/corpora/liet4blerinu2/math_corp/documents/Closure%20operator',
  '/corpora/liet4blerinu2/math_corp/documents/Combinatorics',
  '/corpora/liet4blerinu2/math_corp/documents/Commutative%20algebra',
  '/corpora/liet4blerinu2/math_corp/documents/Compass-and-straightedge%20construction',
  '/corpora/liet4blerinu2/math_corp/d

###Get content of document

In [110]:
subject = "Sine"
url = 'https://gateway.watsonplatform.net/concept-insights/api/v2/corpora/liet4blerinu2/math_corp/documents/' + subject
r = requests.get(url,auth=auth)

print (r.status_code)

ast.literal_eval(r.content)

200


{'id': '/corpora/liet4blerinu2/math_corp/documents/Sine',
 'label': 'Sine',
 'last_modified': '2015-12-10T15:56:15.875Z',
 'parts': [{'content-type': 'text',
   'data': '{{Infobox mathematics function\n| name = Sine\n| image = Sine one period.svg\n| parity=odd |domain=(\xe2\x88\x92,)\\u0026thinsp; |codomain=[\xe2\x88\x921,1]\\u0026thinsp; |period=2\n| zero=0 |plusinf= |minusinf= |max=((2k + \xc2\xbd) 1)\\u0026thinsp; |min=((2k \xe2\x88\x92 \xc2\xbd) \xe2\x88\x921)\n| vr1= |f1=  |vr5= |f5=\n| asymptote= |root=k |critical=k \xe2\x88\x92  |inflection=k |fixed=0\n| notes = \n}}\n\nSine, in mathematics, is a trigonometric function of an angle. The sine of an angle is defined in the context of a right triangle: for the specified angle, it is the ratio of the length of the side that is opposite that angle to the length of the longest side of the triangle (i.e., the hypotenuse).\n\nTrigonometric functions are commonly defined as ratios of two sides of a right triangle containing the angle, and

###Delete document from corpus by id

In [29]:
# DELETE /v2/corpora/{account_id}/{corpus}/documents/{document}

! curl -u "82a698f0-8d8c-42f9-9c7d-1e4b97eebd52":'50KJG6jdgZ6h'\
       -X DELETE 'https://gateway.watsonplatform.net/concept-insights/api/v2/corpora/liet4blerinu2/math_corp/documents/testDocument'

###Stats on concepts present in corpora 

In [80]:
# GET /v2/corpora/{account_id}/{corpus}/stats
url = 'https://gateway.watsonplatform.net/concept-insights/api/v2/corpora/liet4blerinu2/math_corp/stats'
r = requests.get(url,auth=auth)
r.status_code

200

In [6]:
# stats on concepts present in corpora 
ast.literal_eval(r.content)

###Get Graph id 

In [10]:
url = 'https://gateway.watsonplatform.net/concept-insights/api/v2/graphs'
#url = "https://watson-api-explorer.mybluemix.net/concept-insights/api/v2/graphs"
r = requests.get(url ,auth=auth)
r.status_code

200

In [11]:
ast.literal_eval(r.content)

{'graphs': ['/graphs/wikipedia/en-20120601', '/graphs/wikipedia/en-latest']}

In [12]:
graph = ast.literal_eval(r.content).values()[0][0]

###Label Search

In [96]:
# GET /v2/graphs/{account_id}/{graph}/label_search
!curl -u '82a698f0-8d8c-42f9-9c7d-1e4b97eebd52':'50KJG6jdgZ6h''https://gateway.watsonplatform.net/concept-insights/api/v2/graphs/liet4blerinu2/wikipedia/en-latest/label_search?query=math'

curl: no URL specified!
curl: try 'curl --help' or 'curl --manual' for more information


In [95]:
url = 'https://gateway.watsonplatform.net/concept-insights/api'+\
 '/v2/graphs/wikipedia/en-20120601/label_search?query=quadratic'
r = requests.get(url ,auth=auth)
r.status_code

200

In [195]:
ast.literal_eval(r.content)

{'matches': [{'id': '/graphs/wikipedia/en-20120601/concepts/Quadratic',
   'label': 'Quadratic'}]}

In [158]:
#GET /v2/corpora/{account_id}/{corpus}/conceptual_search
url = 'https://gateway.watsonplatform.net/concept-insights/api'+\
 '/v2/corpora/liet4blerinu2/math_corp/conceptual_search?concept_fields=["sine"]'
r  = requests.get(url, auth=auth)
r.status_code

400

In [159]:
r.content #"/graphs/wikipedia/en-20120601/concepts/Slope

'{"error":"missing ids parameter","url":"/api/v2/corpora/liet4blerinu2/math_corp/conceptual_search"}\n'

###Concepts related to a specific article

In [63]:
#GET /v2/corpora/{account_id}/{corpus}/documents/{document}
# Calculus
q = 'Geometry'
#url = 'https://gateway.watsonplatform.net/concept-insights/api'+\
 #     '/v2/corpora/liet4blerinu2/math_corp/documents/' + q + '/related_concepts'
    
url = 'https://gateway.watsonplatform.net/concept-insights/api'+\
      '/v2/corpora/liet4blerinu2/math_corp/documents/' + q +'/related_concepts'

r  = requests.get(url, auth=auth)
r.status_code

400

In [58]:
ast.literal_eval(r.content)

{'id': '/corpora/liet4blerinu2/math_corp/documents/Geometry',
 'label': 'Geometry',
 'last_modified': '2015-12-10T10:07:53.38Z',
 'parts': [{'content-type': 'text',
   'data': 'Geometry (from the \xce\xb3\xce\xb5\xcf\x89\xce\xbc\xce\xb5\xcf\x84\xcf\x81\xce\xaf\xce\xb1; geo- "earth", -metron "measurement") is a branch of mathematics concerned with questions of shape, size, relative position of figures, and the properties of space. A mathematician who works in the field of geometry is called a geometer. Geometry arose independently in a number of early cultures as a body of practical knowledge concerning lengths, areas, and volumes, with elements of formal mathematical science emerging in the West as early as Thales (6th century BC).  By the 3rd century BC, geometry was put into an axiomatic form by Euclid, whose treatment\xe2\x80\x94Euclidean geometry\xe2\x80\x94set a standard for many centuries to follow. Archimedes developed ingenious techniques for calculating areas and volumes, in m

In [8]:
url = 'https://gateway.watsonplatform.net/concept-insights/api'+\
      '/v2/corpora/liet4blerinu2/math_corp/documents/Geometry/related_concepts'
r  = requests.get(url, auth=auth)
r.status_code

400

In [9]:
ast.literal_eval(r.content)

{'error': 'document not ready for querying: document empty or still processing',
 'url': '/api/v2/corpora/liet4blerinu2/math_corp/documents/Geometry/related_concepts'}

##How Related is Radian (Concept) to Calculus (Document)

In [97]:
# GET /v2/corpora/{account_id}/{corpus}/documents/{document}/relation_scores
# concept used has to be present in the concept graph -- otherwise error will be given
concept = "/graphs/wikipedia/en-20120601/concepts/holomorphic embedding"

url = 'https://gateway.watsonplatform.net/concept-insights/api'+\
      '/v2/corpora/liet4blerinu2/math_corp/documents/Complex%20manifold/'+\
      'relation_scores?concepts=["/graphs/wikipedia/en-20120601/concepts/"]'
        
r  = requests.get(url, auth=auth)
r.status_code

200

In [98]:
ast.literal_eval(r.content) # "/graphs/wikipedia/en-20120601/concepts/Radian"

{'scores': [{'concept': '/graphs/wikipedia/en-20120601/concepts/',
   'score': 0.5}]}

##Build Smart Cards Data Pipeline

##CI will extract concepts from block of text - but contains False Negatives 

In [13]:
text = '''Mathematics 117, Trigonometry, helps students develop skills sufficiently to write and
use the definition of trigonometric functions; sketch the graph of the trigonometric
functions; prove identities; solve trigonometric equations; learn and then apply the law
of the sines and cosines; learn how to write a complex number in trigonometric form
and find all the roots of a complex number; learn polar coordinates system and the
graphs of some simple equations in polar; learn about conic sections (rectangular &
polar), vector (applications & operations), and the exponential and logarithmic
functions with applications and modeling. A calculator is required.
- Applicable toward graduation where program structure permits:
- Certificate or Degree - All Certificates and All Degrees
- Group Requirement - Mathematics (A.A.S. only)
- Area of Concentration - Mathematics '''

In [104]:
url='https://gateway.watsonplatform.net/concept-insights/api/v2/graphs/wikipedia/en-20120601/annotate_text'
headers = { 'Content-Type': 'text/plain'}
r = requests.post(url, 
                  headers=headers, 
                  data={'body': text},
                  auth=auth)

print (r.status_code)
concepts = set(label['concept']['label']for label in ast.literal_eval(r.content).values()[0])

copy = concepts.copy()
pattern1 = '[0-9]'
pattern2 = '\(\w+\)'

for concept in concepts:
    # drop concepts with numerics
    r = re.search(pattern1,concept)
    if r:
        copy.discard(concept)
        continue
        
    r = re.search(pattern2,concept)
    if r:
        copy.discard(concept)
        
results = list(copy)

201
set(['Logarithm', 'Addition', 'Complex number', 'Equation', 'Graduation', 'Mathematics', 'Percentage', 'Trigonometry', 'Calculator', 'Conic section', 'Euclidean vector', 'Student', 'Computer program', 'Apply', 'Law', 'Trigonometric functions', 'Nth root'])


In [105]:
list(copy)

['Logarithm',
 'Addition',
 'Complex number',
 'Equation',
 'Graduation',
 'Mathematics',
 'Percentage',
 'Trigonometry',
 'Calculator',
 'Conic section',
 'Euclidean vector',
 'Student',
 'Computer program',
 'Apply',
 'Law',
 'Trigonometric functions',
 'Nth root']

In [80]:
content = 'Identity (mathematics)'
pattern = '\(\w+\)'
r = re.search(pattern,content)
content[:r.start()] + content[r.end():]

u'Identity '

In [90]:
content = 'Identity (mathematics)'
pattern = '([\w]+)\ (\(\w+\))'
match = re.search(pattern,content)
if match:
    print (match.group(1))


Identity


In [95]:
content = '29 (number)'
pattern = '[0-9]'
r = re.match(pattern,content)

In [96]:
type(r)

_regex.Match

In [100]:
copy = concepts.copy()
pattern1 = '[0-9]'
pattern2 = '\(\w+\)'

for concept in concepts:
    # drop concepts with numerics
    r = re.search(pattern1,concept)
    if r:
        copy.discard(concept)
        continue
        
    r = re.search(pattern2,concept)
    if r:
        copy.discard(concept)
        

In [101]:
copy

{'Addition',
 'Apply',
 'Calculator',
 'Complex number',
 'Computer program',
 'Conic section',
 'Equation',
 'Euclidean vector',
 'Graduation',
 'Law',
 'Logarithm',
 'Mathematics',
 'Nth root',
 'Percentage',
 'Student',
 'Trigonometric functions',
 'Trigonometry'}

In [21]:
import regex as re
from collections import defaultdict

In [ ]:
# GET /v2/corpora/{account_id}/{corpus}/documents
limit = '0'
url = 'https://gateway.watsonplatform.net/concept-insights/api/v2/corpora/liet4blerinu2/math_corp/documents?limit=' + limit
r = requests.get(url,auth=auth)
docs = ast.literal_eval(r.content)

In [16]:
results = defaultdict(list)
for doc in docs['documents']:
    match = re.search(r'documents/([\w \d%]+)', doc)
    if match: 
        doc_name = match.group(1) #document name

    #Retrieves concepts related to a specific article
    url = 'https://gateway.watsonplatform.net/concept-insights/api'+\
          '/v2/corpora/liet4blerinu2/math_corp/documents/'+\
           doc_name + '/related_concepts'
    r  = requests.get(url, auth=auth)
    ci = ast.literal_eval(r.content) # list of concepts from concept insights per document

    #Creates a dictionary with concepts as keys, (doc_name, score) as values
    print (r.status_code)
    for i, d in enumerate(ci['concepts']):
        c = d['concept']['label']
        s = d['score']
        results[c].append((doc_name, s))
        print (i)

In [ ]:
url='https://gateway.watsonplatform.net/concept-insights/api/v2'+ graph +'/annotate_text'
headers = { 'Content-Type': 'text/plain'}
r = requests.post(url, 
                  headers=headers, 
                  data={'body': text},
                  auth=auth)
r.status_code

In [297]:
# GET /v2/graphs/{account_id}/{graph}/concepts/{concept}/related_concepts
url = 'https://gateway.watsonplatform.net/concept-insights/api/v2/graphs/wikipedia/en-20120601/concepts/Topology/related_concepts'
r = requests.get(url,
               auth=auth)
r.status_code

200

In [298]:
ast.literal_eval(r.content)

{'concepts': [{'concept': {'id': '/graphs/wikipedia/en-20120601/concepts/Topology',
    'label': 'Topology'},
   'score': 0.9999561},
  {'concept': {'id': '/graphs/wikipedia/en-20120601/concepts/Glossary_of_topology',
    'label': 'Glossary of topology'},
   'score': 0.99205375},
  {'concept': {'id': '/graphs/wikipedia/en-20120601/concepts/Hausdorff_space',
    'label': 'Hausdorff space'},
   'score': 0.99138063},
  {'concept': {'id': '/graphs/wikipedia/en-20120601/concepts/Discrete_space',
    'label': 'Discrete space'},
   'score': 0.98841244},
  {'concept': {'id': '/graphs/wikipedia/en-20120601/concepts/Topological_space',
    'label': 'Topological space'},
   'score': 0.98813605},
  {'concept': {'id': '/graphs/wikipedia/en-20120601/concepts/Locally_compact_space',
    'label': 'Locally compact space'},
   'score': 0.9845779},
  {'concept': {'id': '/graphs/wikipedia/en-20120601/concepts/Connected_space',
    'label': 'Connected space'},
   'score': 0.9829185},
  {'concept': {'id': '

In [110]:
f = open("./bluemix_authentication.csv", "r")
ast.literal_eval(f.read())

('82a698f0-8d8c-42f9-9c7d-1e4b97eebd52', '50KJG6jdgZ6h')